In [1]:
import numpy as np
import os

# Function to load embeddings from saved files
def load_embeddings(directory):
    # Extract numeric ranges and sort filenames accordingly
    def extract_range(filename):
        start, end = map(int, filename.rstrip('.npy').split('_')[-2:])
        return start, end
    
    # Get sorted filenames
    sentence_embeddings_files = sorted([f for f in os.listdir(directory) if f.startswith('sentence_embeddings_') and f.endswith('.npy')],
                                       key=extract_range)
    text_embeddings_files = sorted([f for f in os.listdir(directory) if f.startswith('text_embeddings_') and f.endswith('.npy')],
                                   key=extract_range)
    
    sentence_embeddings_list = []
    text_embeddings_list = []
    
    for sentence_file, text_file in zip(sentence_embeddings_files, text_embeddings_files):
        print(f"Loading sentence embeddings from file: {sentence_file}")
        sentence_embeddings = np.load(os.path.join(directory, sentence_file))
        
        print(f"Loading text embeddings from file: {text_file}")
        text_embeddings = np.load(os.path.join(directory, text_file))
        
        sentence_embeddings_list.append(sentence_embeddings)
        text_embeddings_list.append(text_embeddings)
    
    all_sentence_embeddings = np.concatenate(sentence_embeddings_list, axis=0)
    all_text_embeddings = np.concatenate(text_embeddings_list, axis=0)
    
    return all_sentence_embeddings, all_text_embeddings

# Load embeddings for training
all_sentence_embeddings, all_text_embeddings = load_embeddings('bert_embeddings')
print("All Sentence Embeddings Shape:", all_sentence_embeddings.shape)
print("All Text Embeddings Shape:", all_text_embeddings.shape)



Loading sentence embeddings from file: sentence_embeddings_1_1000.npy
Loading text embeddings from file: text_embeddings_1_1000.npy
Loading sentence embeddings from file: sentence_embeddings_1001_2000.npy
Loading text embeddings from file: text_embeddings_1001_2000.npy
Loading sentence embeddings from file: sentence_embeddings_2001_3000.npy
Loading text embeddings from file: text_embeddings_2001_3000.npy
Loading sentence embeddings from file: sentence_embeddings_3001_4000.npy
Loading text embeddings from file: text_embeddings_3001_4000.npy
Loading sentence embeddings from file: sentence_embeddings_4001_5000.npy
Loading text embeddings from file: text_embeddings_4001_5000.npy
Loading sentence embeddings from file: sentence_embeddings_5001_6000.npy
Loading text embeddings from file: text_embeddings_5001_6000.npy
Loading sentence embeddings from file: sentence_embeddings_6001_7000.npy
Loading text embeddings from file: text_embeddings_6001_7000.npy
Loading sentence embeddings from file: s

In [2]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import os

# Load labels from CSV file
labels_df = pd.read_csv('preprocessed_data.csv')
labels = labels_df['class'].values
print("Labels Shape:", labels.shape)

C:\Users\bindu\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Labels Shape: (24751,)


In [3]:
# Define the neural network architecture
max_sentences = all_sentence_embeddings.shape[1]  # Number of sentences per sample
embedding_dim = all_sentence_embeddings.shape[2]  # Embedding dimension

print(max_sentences)
print(embedding_dim)

6
768


In [4]:
# Convert labels to numpy array
labels = np.array(labels)
all_sentence_embeddings = np.array(all_sentence_embeddings)
all_text_embeddings = np.array(all_text_embeddings)

In [5]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from sklearn.model_selection import train_test_split

# Define the necessary parameters
num_sentences = 6  # Replace with your actual number of sentences per sample
embedding_dim = 768  # Dimension of the BERT embeddings
num_classes = 4  # Number of classes for classification

# Input shape for sentence embeddings
input_sentences = Input(shape=(num_sentences, embedding_dim))

# Define parallel lines for sentence embeddings
sentence_outputs = []
for _ in range(num_sentences):
    x = Dense(128, activation='relu')(input_sentences[:, _, :])
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    sentence_output = Dense(20, activation='relu')(x)
    sentence_outputs.append(sentence_output)

# Input shape for text embeddings
input_text = Input(shape=(embedding_dim,))

# Define parallel lines for text embeddings
text_outputs = []
for _ in range(3):
    x = Dense(128, activation='relu')(input_text)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    text_output = Dense(60, activation='relu')(x)
    text_outputs.append(text_output)

# Concatenate the outputs of sentence and text branches
concatenated = Concatenate()(sentence_outputs + text_outputs)

# Define a fully connected layer
x = Dense(256, activation='relu')(concatenated)
x = Dropout(0.2)(x)

# Define the sequential layers
sequential_layers = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

# Connect the fully connected layer with the sequential layers
output = sequential_layers(x)

# Build the model
model = Model(inputs=[input_sentences, input_text], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 6, 768)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 768)       │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 768)       │          0 │ input_layer[0][0] │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 768)       │          0 │ input_layer[0][0] │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_3          │ (None, 768)       │          0 │ input_layer[0][0] │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_4          │ (None, 768)       │          0 │ input_layer[0][0] │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_5          │ (None, 768)       │          0 │ input_layer[0][0] │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     98,432 │ get_item[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     98,432 │ get_item_1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     98,432 │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 128)       │     98,432 │ get_item_3[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 128)       │     98,432 │ get_item_4[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 128)       │     98,432 │ get_item_5[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 128)       │     98,432 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 128)       │     98,432 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 128)       │     98,432 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 128)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 128)       │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 128)       │          0 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,134,448 (4.33 MB)

 Trainable params: 1,134,448 (4.33 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:

num_samples = 24751
num_sentences = 6 
embedding_dim = 768  

X_sentence_embeddings = all_sentence_embeddings  # This should be of shape (num_samples, num_sentences, embedding_dim)
X_text_embeddings = all_text_embeddings  # This should be of shape (num_samples, embedding_dim)
y_labels = labels  # This should be of shape (num_samples,)

# Ensure your data is the correct shape
assert X_sentence_embeddings.shape == (num_samples, num_sentences, embedding_dim)
assert X_text_embeddings.shape == (num_samples, embedding_dim)
assert y_labels.shape[0] == num_samples

# Split data into training and validation sets
X_train_sentences, X_val_sentences, X_train_text, X_val_text, y_train, y_val = train_test_split(
    X_sentence_embeddings, X_text_embeddings, y_labels, test_size=0.2, random_state=42
)

# Train the model
history = model.fit(
    [X_train_sentences, X_train_text],
    y_train,
    validation_data=([X_val_sentences, X_val_text], y_val),
    epochs=40,  # Set the number of epochs
    batch_size=32,  # Set the batch size
    verbose=1  # Print progress during training
)


Epoch 1/40


C:\Users\bindu\AppData\Roaming\Python\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_37']. Received: the structure of inputs=('*', '*')
  warnings.warn(


619/619 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7977 - loss: 0.5820 - val_accuracy: 0.8503 - val_loss: 0.4108
Epoch 2/40
619/619 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8478 - loss: 0.4358 - val_accuracy: 0.8524 - val_loss: 0.3932
Epoch 3/40
619/619 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8518 - loss: 0.4104 - val_accuracy: 0.8562 - val_loss: 0.3873
Epoch 4/40
619/619 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8574 - loss: 0.3874 - val_accuracy: 0.8574 - val_loss: 0.3771
Epoch 5/40
619/619 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8599 - loss: 0.3856 - val_accuracy: 0.8582 - val_loss: 0.3811
Epoch 6/40
619/619 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8670 - loss: 0.3626 - val_accuracy: 0.8503 - val_loss: 0.4176
Epoch 7/40
619/619 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8670 - loss: 0.3628 - val_accuracy: 0.8604 - val_loss: 0.3861
Epoch 8/40
619/619 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8758 - loss: 0.3375 - val_accuracy: 0.8566 - val_

In [ ]:

# Prepare binary labels for Stage 1: normal text
labels_binary = np.where(labels == 2, 0, 1)

# Separate humor samples for Stage 2 classification 
humor_labels = labels[labels != 2]  # exclude neither category
humor_sentence_embeddings = all_sentence_embeddings[labels != 2]
humor_text_embeddings = all_text_embeddings[labels != 2]

# Create binary labels for Stage 2 (0 for hate, 1 for offensive)
humor_labels_binary = np.where(humor_labels == 0, 0, 1)


In [ ]:

from tensorflow.keras.layers import Dense, Dropout, Input, Concatenate
from tensorflow.keras.models import Model

# Stage 1:
input_sentences = Input(shape=(num_sentences, embedding_dim))
input_text = Input(shape=(embedding_dim,))

# Parallel layers for sentence and text embeddings
sentence_layers = [Dense(128, activation='relu')(input_sentences[:, i, :]) for i in range(num_sentences)]
text_layers = [Dense(128, activation='relu')(input_text)] * 3  # Repeating for consistency

# Concatenate layers
combined = Concatenate()(sentence_layers + text_layers)

# Fully connected layers
x = Dense(256, activation='relu')(combined)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output_stage1 = Dense(1, activation='sigmoid')(x)

# Model for Stage 1
stage1_model = Model(inputs=[input_sentences, input_text], outputs=output_stage1)
stage1_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
stage1_model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 6, 768)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_6          │ (None, 768)       │          0 │ input_layer_3[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_7          │ (None, 768)       │          0 │ input_layer_3[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_8          │ (None, 768)       │          0 │ input_layer_3[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_9          │ (None, 768)       │          0 │ input_layer_3[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_10         │ (None, 768)       │          0 │ input_layer_3[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_11         │ (None, 768)       │          0 │ input_layer_3[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_31 (Dense)    │ (None, 128)       │     98,432 │ get_item_6[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_32 (Dense)    │ (None, 128)       │     98,432 │ get_item_7[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 128)       │     98,432 │ get_item_8[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_34 (Dense)    │ (None, 128)       │     98,432 │ get_item_9[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_35 (Dense)    │ (None, 128)       │     98,432 │ get_item_10[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 128)       │     98,432 │ get_item_11[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_37 (Dense)    │ (None, 128)       │     98,432 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 1152)      │          0 │ dense_31[0][0],   │
│ (Concatenate)       │                   │            │ dense_32[0][0],   │
│                     │                   │            │ dense_33[0][0],   │
│                     │                   │            │ dense_34[0][0],   │
│                     │                   │            │ dense_35[0][0],   │
│                     │                   │            │ dense_36[0][0],   │
│                     │                   │            │ dense_37[0][0],   │
│                     │                   │            │ dense_37[0][0],   │
│                     │                   │            │ dense_37[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_38 (Dense)    │ (None, 256)       │    295,168 │ concatenate_1[0]

 Total params: 1,017,217 (3.88 MB)

 Trainable params: 1,017,217 (3.88 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:

from sklearn.model_selection import train_test_split

# Split data for Stage 1
X_train_sen, X_val_sen, X_train_text, X_val_text, y_train_bin, y_val_bin = train_test_split(
    all_sentence_embeddings, all_text_embeddings, labels_binary, test_size=0.2, random_state=42
)

# Train Stage 1 model
history_stage1 = stage1_model.fit(
    [X_train_sen, X_train_text],
    y_train_bin,
    validation_data=([X_val_sen, X_val_text], y_val_bin),
    epochs=20,
    batch_size=32,
    verbose=1
)


Epoch 1/20


C:\Users\bindu\AppData\Roaming\Python\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_63', 'keras_tensor_64']. Received: the structure of inputs=('*', '*')
  warnings.warn(


619/619 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.8696 - loss: 0.3094 - val_accuracy: 0.8950 - val_loss: 0.2339
Epoch 2/20
619/619 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9003 - loss: 0.2273 - val_accuracy: 0.9035 - val_loss: 0.2176
Epoch 3/20
619/619 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9133 - loss: 0.2035 - val_accuracy: 0.9119 - val_loss: 0.2052
Epoch 4/20
619/619 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9173 - loss: 0.1922 - val_accuracy: 0.9111 - val_loss: 0.2066
Epoch 5/20
619/619 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9231 - loss: 0.1768 - val_accuracy: 0.9075 - val_loss: 0.2245
Epoch 6/20
619/619 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9308 - loss: 0.1612 - val_accuracy: 0.8950 - val_loss: 0.2412
Epoch 7/20
619/619 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9358 - loss: 0.1493 - val_accuracy: 0.9059 - val_loss: 0.2250
Epoch 8/20
619/619 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9376 - loss: 0.1434 - val_accuracy: 0.9073 -

In [ ]:

# Stage 2: 
input_sentences_stage2 = Input(shape=(num_sentences, embedding_dim))
input_text_stage2 = Input(shape=(embedding_dim,))

# Layers for Stage 2
sentence_layers_stage2 = [Dense(128, activation='relu')(input_sentences_stage2[:, i, :]) for i in range(num_sentences)]
text_layers_stage2 = [Dense(128, activation='relu')(input_text_stage2)] * 3

# Concatenate layers
combined_stage2 = Concatenate()(sentence_layers_stage2 + text_layers_stage2)

# Fully connected layers
x_stage2 = Dense(256, activation='relu')(combined_stage2)
x_stage2 = Dropout(0.2)(x_stage2)
x_stage2 = Dense(128, activation='relu')(x_stage2)
x_stage2 = Dropout(0.2)(x_stage2)
output_stage2 = Dense(1, activation='sigmoid')(x_stage2)

# Model for Stage 2
stage2_model = Model(inputs=[input_sentences_stage2, input_text_stage2], outputs=output_stage2)
stage2_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
stage2_model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 6, 768)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_12         │ (None, 768)       │          0 │ input_layer_5[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_13         │ (None, 768)       │          0 │ input_layer_5[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_14         │ (None, 768)       │          0 │ input_layer_5[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_15         │ (None, 768)       │          0 │ input_layer_5[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_16         │ (None, 768)       │          0 │ input_layer_5[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_17         │ (None, 768)       │          0 │ input_layer_5[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_41 (Dense)    │ (None, 128)       │     98,432 │ get_item_12[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_42 (Dense)    │ (None, 128)       │     98,432 │ get_item_13[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 128)       │     98,432 │ get_item_14[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_44 (Dense)    │ (None, 128)       │     98,432 │ get_item_15[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_45 (Dense)    │ (None, 128)       │     98,432 │ get_item_16[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_46 (Dense)    │ (None, 128)       │     98,432 │ get_item_17[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_47 (Dense)    │ (None, 128)       │     98,432 │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 1152)      │          0 │ dense_41[0][0],   │
│ (Concatenate)       │                   │            │ dense_42[0][0],   │
│                     │                   │            │ dense_43[0][0],   │
│                     │                   │            │ dense_44[0][0],   │
│                     │                   │            │ dense_45[0][0],   │
│                     │                   │            │ dense_46[0][0],   │
│                     │                   │            │ dense_47[0][0],   │
│                     │                   │            │ dense_47[0][0],   │
│                     │                   │            │ dense_47[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_48 (Dense)    │ (None, 256)       │    295,168 │ concatenate_2[0]

 Total params: 1,017,217 (3.88 MB)

 Trainable params: 1,017,217 (3.88 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:

# Split data for Stage 2
X_train_sen_humor, X_val_sen_humor, X_train_text_humor, X_val_text_humor, y_train_humor, y_val_humor = train_test_split(
    humor_sentence_embeddings, humor_text_embeddings, humor_labels_binary, test_size=0.2, random_state=42
)

# Train Stage 2 model
history_stage2 = stage2_model.fit(
    [X_train_sen_humor, X_train_text_humor],
    y_train_humor,
    validation_data=([X_val_sen_humor, X_val_text_humor], y_val_humor),
    epochs=40,
    batch_size=32,
    verbose=1
)


Epoch 1/40


C:\Users\bindu\AppData\Roaming\Python\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_84', 'keras_tensor_85']. Received: the structure of inputs=('*', '*')
  warnings.warn(


516/516 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - accuracy: 0.9275 - loss: 0.2571 - val_accuracy: 0.9304 - val_loss: 0.2013
Epoch 2/40
516/516 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9301 - loss: 0.2050 - val_accuracy: 0.9304 - val_loss: 0.1935
Epoch 3/40
516/516 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9319 - loss: 0.1934 - val_accuracy: 0.9330 - val_loss: 0.2159
Epoch 4/40
516/516 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9322 - loss: 0.1976 - val_accuracy: 0.9321 - val_loss: 0.1951
Epoch 5/40
516/516 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9382 - loss: 0.1887 - val_accuracy: 0.9301 - val_loss: 0.1982
Epoch 6/40
516/516 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9385 - loss: 0.1797 - val_accuracy: 0.9340 - val_loss: 0.1971
Epoch 7/40
516/516 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9407 - loss: 0.1670 - val_accuracy: 0.9308 - val_loss: 0.2000
Epoch 8/40
516/516 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9485 - loss: 0.1542 - val_accuracy: 0.9250

In [11]:
from sklearn.metrics import classification_report, confusion_matrix

# Step 1: Predict on the validation set
y_pred_probs = stage2_model.predict([X_val_sen_humor, X_val_text_humor])

# Step 2: Convert probabilities to binary labels
y_pred_binary = (y_pred_probs > 0.5).astype(int)

# Step 3: Compute the confusion matrix
conf_matrix = confusion_matrix(y_val_humor, y_pred_binary)

# Step 4: Compute metrics for each class
report = classification_report(y_val_humor, y_pred_binary, target_names=["Class 0", "Class 1"])

# Print results
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(report)

129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Confusion Matrix:
[[  68  220]
 [ 129 3704]]

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.35      0.24      0.28       288
     Class 1       0.94      0.97      0.96      3833

    accuracy                           0.92      4121
   macro avg       0.64      0.60      0.62      4121
weighted avg       0.90      0.92      0.91      4121

